In [1]:
%matplotlib inline

In [2]:
import os, sys
current_dir = os.getcwd()

In [106]:
pwd

u'/home/ubuntu/nbs/fisheries/train'

In [4]:
HOME_DIR = '/home/ubuntu/nbs/fisheries'

TRAIN_DIR = HOME_DIR + '/train'
VALID_DIR = HOME_DIR + '/valid'
RESULTS_DIR = HOME_DIR + '/results'
SAMPLE_DIR = HOME_DIR + '/sample'

In [104]:
os.mkdir('sample')
os.mkdir('results')
os.mkdir('valid')
os.mkdir('sample/test')
os.mkdir('sample/train')
os.mkdir('sample/valid')
os.mkdir('test_stg1/unknown')
os.mkdir('test_stg2/unknown')

OSError: [Errno 2] No such file or directory: 'test_stg1/unknown'

In [ ]:
%cd test_stg1/
%mv *.jpg unknown/

In [66]:
ls unknown/

img_00005.jpg  img_01571.jpg  img_03261.jpg  img_04811.jpg  img_06321.jpg
img_00007.jpg  img_01593.jpg  img_03270.jpg  img_04814.jpg  img_06324.jpg
img_00009.jpg  img_01595.jpg  img_03283.jpg  img_04830.jpg  img_06332.jpg
img_00018.jpg  img_01616.jpg  img_03290.jpg  img_04835.jpg  img_06345.jpg
img_00027.jpg  img_01618.jpg  img_03297.jpg  img_04856.jpg  img_06346.jpg
img_00030.jpg  img_01627.jpg  img_03310.jpg  img_04860.jpg  img_06349.jpg
img_00040.jpg  img_01632.jpg  img_03312.jpg  img_04870.jpg  img_06368.jpg
img_00046.jpg  img_01633.jpg  img_03324.jpg  img_04872.jpg  img_06369.jpg
img_00053.jpg  img_01640.jpg  img_03333.jpg  img_04879.jpg  img_06371.jpg
img_00071.jpg  img_01646.jpg  img_03342.jpg  img_04885.jpg  img_06373.jpg
img_00075.jpg  img_01647.jpg  img_03346.jpg  img_04888.jpg  img_06401.jpg
img_00102.jpg  img_01649.jpg  img_03355.jpg  img_04921.jpg  img_06412.jpg
img_00103.jpg  img_01657.jpg  img_03361.jpg  img_04949.jpg  img_06424.jpg
img_00109.jpg  img_01669.jpg  img_0337

In [6]:
#Allow relative imports to directories above lesson1/
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# Imports

In [7]:
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l2, activity_l2, l1, activity_l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.layer_utils import layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


In [8]:
import os
from glob import glob
import pandas as pd
import shutil
import numpy as np



In [9]:
import random

## Make Sample/Valid sets

In [68]:
cd ../train/

/home/ubuntu/nbs/fisheries/train


In [69]:
ls

ALB/  BET/  DOL/  LAG/  NoF/  OTHER/  SHARK/  YFT/


In [ ]:
# run this in valid/, sample/valid, sample/train,
os.mkdir('ALB')
os.mkdir('BET')
os.mkdir('DOL')
os.mkdir('LAG')
os.mkdir('NoF')
os.mkdir('OTHER')
os.mkdir('SHARK')
os.mkdir('YFT')

In [10]:
cd ../valid/

[Errno 2] No such file or directory: '../valid/'
/home/ubuntu/nbs/fisheries


In [54]:
cd ../sample/valid/

/home/ubuntu/nbs/fisheries/sample/valid


In [55]:
cd ../train/

/home/ubuntu/nbs/fisheries/sample/train


In [22]:
# CD to train/
# Move 20% of the files in each train/ directory to the corresponding valid/ directory
for paths, dirs, files in os.walk("."):
    for dir in dirs:
        g = glob(dir+'/*.jpg')
        validSize = int(len(g) * .2) #20% of the files in each training folder
        shuf = np.random.permutation(g)
#         print dir
#         print (len(g))
#         print validSize
        for i in range(validSize): 
            shutil.move(shuf[i], '../valid/' + shuf[i])
            
        
            

DOL
94
18
YFT
588
117
BET
160
32
LAG
54
10
OTHER
240
48
NoF
372
74
ALB
1376
275
SHARK
141
28


In [50]:
# CD to train/
# copy 20% of the files from train to their respective sample folder
g = glob('*/*.jpg')
sampleTrainLen = int(len(g) * .2)
print sampleTrainLen
shuf = np.random.permutation(g)
for i in range(sampleTrainLen): shutil.copyfile(shuf[i], '../sample/train/' + shuf[i])

605


In [53]:
# CD to valid/
# copy 20% of the files from valid to their respective sample folder
g = glob('*/*.jpg')
sampleValidLen = int(len(g) * .2)
print sampleValidLen
shuf = np.random.permutation(g)
for i in range(sampleValidLen): shutil.copyfile(shuf[i], '../sample/valid/' + shuf[i])

150


## Load Batches

In [87]:
path= "/home/ubuntu/nbs/fisheries/"
# path = "/home/ubuntu/nbs/fisheries/sample/"

In [88]:
batch_size=64

In [89]:
# Making Data Augmentation Generator
daGen = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

In [90]:
# Initializing A Generator
gen = image.ImageDataGenerator()

In [91]:
# Setting Training Batches
batches = daGen.flow_from_directory(path+'train',target_size=(224,224),
                                    class_mode='categorical', shuffle=True, batch_size=batch_size)

Found 3025 images belonging to 8 classes.


In [92]:
#Setting Valid Batches
val_batches = gen.flow_from_directory(path+'valid',target_size=(224,224), 
                                        class_mode='categorical', shuffle=False, batch_size=batch_size*2)

Found 752 images belonging to 8 classes.


In [156]:
#Setting Valid Batches
test_batches = gen.flow_from_directory(path+'test_stg1',target_size=(224,224), 
                                        class_mode='categorical', shuffle=False, batch_size=1)

Found 1000 images belonging to 1 classes.


In [110]:
#Save Weights So You Can Return to Them Later 
model.model.save_weights(path+'/results/Fisheries-CNN-Weights.h5')

In [ ]:
#Save Weights So You Can Return to Them Later 
model.model.load_weights(path+'/results/Fisheries-CNN-Weights.h5')

### Initialize Model, Compile, And Train

In [95]:
#Model Layers
model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(16,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D(),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D(),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D(),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dropout(0.5),
            Dense(500, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(8, activation='softmax')
        ])

In [96]:
#Competition is evaluating on multi-categorical log loss so compile with categorical_crossentropy
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [100]:
#Fit (train) Your Model!
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches,
                    nb_val_samples=val_batches.nb_sample)

Epoch 1/5
3025/3025 [==============================] - 95s - loss: 1.6122 - acc: 0.4807 - val_loss: 1.9959 - val_acc: 0.3218
Epoch 2/5
3025/3025 [==============================] - 87s - loss: 1.4851 - acc: 0.5217 - val_loss: 1.5753 - val_acc: 0.5013
Epoch 3/5
3025/3025 [==============================] - 83s - loss: 1.3827 - acc: 0.5514 - val_loss: 1.2574 - val_acc: 0.6250
Epoch 4/5
3025/3025 [==============================] - 82s - loss: 1.3852 - acc: 0.5597 - val_loss: 1.0103 - val_acc: 0.6955
Epoch 5/5
3025/3025 [==============================] - 81s - loss: 1.2171 - acc: 0.6026 - val_loss: 0.8729 - val_acc: 0.7367


In [103]:
model.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                    nb_val_samples=val_batches.nb_sample)

Epoch 1/3
3025/3025 [==============================] - 99s - loss: 0.9649 - acc: 0.6641 - val_loss: 0.6762 - val_acc: 0.7899
Epoch 2/3
3025/3025 [==============================] - 81s - loss: 0.8593 - acc: 0.7058 - val_loss: 0.6341 - val_acc: 0.8112
Epoch 3/3
3025/3025 [==============================] - 80s - loss: 0.7978 - acc: 0.7246 - val_loss: 0.6139 - val_acc: 0.8112


In [107]:
model.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                    nb_val_samples=val_batches.nb_sample)

Epoch 1/3
3025/3025 [==============================] - 94s - loss: 0.7496 - acc: 0.7398 - val_loss: 0.5233 - val_acc: 0.8391
Epoch 2/3
3025/3025 [==============================] - 84s - loss: 0.7144 - acc: 0.7501 - val_loss: 0.5184 - val_acc: 0.8457
Epoch 3/3
3025/3025 [==============================] - 82s - loss: 0.7114 - acc: 0.7428 - val_loss: 0.5153 - val_acc: 0.8444


In [109]:
model.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                    nb_val_samples=val_batches.nb_sample)

Epoch 1/3
3025/3025 [==============================] - 96s - loss: 0.7114 - acc: 0.7511 - val_loss: 0.5107 - val_acc: 0.8444
Epoch 2/3
3025/3025 [==============================] - 80s - loss: 0.6681 - acc: 0.7633 - val_loss: 0.5083 - val_acc: 0.8457
Epoch 3/3
3025/3025 [==============================] - 82s - loss: 0.7361 - acc: 0.7461 - val_loss: 0.5057 - val_acc: 0.8484


## Predictions And Submission

In [157]:
# You need to get predictions from both test files and concat them to submit
predictions = model.predict_generator(test_batches, test_batches.nb_sample)

In [158]:
predictions

array([[  9.09740031e-01,   8.71100277e-03,   6.04057568e-04, ...,
          1.30768947e-03,   1.30786459e-04,   7.01984838e-02],
       [  5.65872729e-01,   2.02152841e-02,   1.82989542e-03, ...,
          1.83657095e-01,   1.45587564e-01,   5.78121059e-02],
       [  3.56708318e-02,   3.16427741e-03,   5.70101943e-03, ...,
          2.16236833e-04,   4.09923017e-01,   8.59838328e-04],
       ..., 
       [  4.65520397e-02,   1.20297610e-03,   9.04971967e-05, ...,
          9.34250355e-01,   1.00855064e-03,   1.17971916e-02],
       [  7.77559698e-01,   1.90431569e-02,   2.86960602e-02, ...,
          8.99930149e-02,   1.30716094e-03,   2.07220055e-02],
       [  2.11675018e-01,   1.13346003e-01,   5.29158674e-03, ...,
          2.94380307e-01,   1.13888420e-02,   3.24251175e-01]], dtype=float32)

In [114]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [159]:
subm = do_clip(predictions,0.93)

In [118]:
subm_csv = path+'results/fisheries-subm.csv'

In [120]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)

In [161]:
submission1 = pd.DataFrame(subm, columns=classes)


In [162]:
submission1.insert(0, 'image', [a[8:] for a in test_batches.filenames])

In [163]:
submission1

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_06237.jpg,0.909740,0.008711,0.007778,0.007778,0.008847,0.007778,0.007778,0.070198
1,img_06893.jpg,0.565873,0.020215,0.007778,0.007778,0.022518,0.183657,0.145588,0.057812
2,img_02082.jpg,0.035671,0.007778,0.007778,0.011104,0.533361,0.007778,0.409923,0.007778
3,img_06261.jpg,0.470010,0.011551,0.007778,0.007778,0.014561,0.411038,0.022642,0.061647
4,img_03628.jpg,0.924974,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.057595
5,img_04860.jpg,0.907143,0.007778,0.007778,0.023822,0.009482,0.048540,0.007778,0.007778
6,img_01640.jpg,0.930000,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778
7,img_02709.jpg,0.930000,0.007778,0.007778,0.007778,0.013231,0.007778,0.007778,0.010024
8,img_00030.jpg,0.642062,0.007778,0.011184,0.007778,0.171608,0.007778,0.099234,0.060132
9,img_04740.jpg,0.008718,0.022462,0.675401,0.007778,0.282868,0.007778,0.007778,0.007778


In [148]:
submission = pd.DataFrame(subm, columns=classes)

In [149]:
submission1.insert(0, 'image', [a[8:] for a in test_batches.filenames])

In [173]:
submission

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,test_stg2/image_05638.jpg,0.241726,0.007778,0.007778,0.007778,0.118301,0.072901,0.020619,0.544040
1,test_stg2/image_02364.jpg,0.292596,0.007778,0.007778,0.007778,0.678242,0.007778,0.007778,0.017432
2,test_stg2/image_10663.jpg,0.018725,0.263522,0.065241,0.007778,0.007778,0.011462,0.026657,0.608163
3,test_stg2/image_00255.jpg,0.110742,0.007778,0.007778,0.007778,0.818831,0.007778,0.007778,0.052947
4,test_stg2/image_08003.jpg,0.241881,0.037752,0.007778,0.007778,0.393834,0.059190,0.021995,0.237437
5,test_stg2/image_05669.jpg,0.344554,0.074545,0.012542,0.007778,0.262711,0.062882,0.024263,0.210736
6,test_stg2/image_11511.jpg,0.574056,0.030472,0.007778,0.007778,0.014284,0.075149,0.026620,0.270510
7,test_stg2/image_10794.jpg,0.007778,0.007778,0.007778,0.007778,0.124734,0.015764,0.045092,0.807639
8,test_stg2/image_04845.jpg,0.230849,0.007778,0.007778,0.007778,0.429894,0.011407,0.007778,0.320754
9,test_stg2/image_02907.jpg,0.732523,0.012688,0.007778,0.007778,0.238167,0.007778,0.007778,0.010752


In [172]:
submission['image'] = 'test_stg2/' + submission['image'].astype(str)

In [174]:
frames = [submission, submission1]

In [175]:
results = pd.concat(frames)

In [176]:
results

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,test_stg2/image_05638.jpg,0.241726,0.007778,0.007778,0.007778,0.118301,0.072901,0.020619,0.544040
1,test_stg2/image_02364.jpg,0.292596,0.007778,0.007778,0.007778,0.678242,0.007778,0.007778,0.017432
2,test_stg2/image_10663.jpg,0.018725,0.263522,0.065241,0.007778,0.007778,0.011462,0.026657,0.608163
3,test_stg2/image_00255.jpg,0.110742,0.007778,0.007778,0.007778,0.818831,0.007778,0.007778,0.052947
4,test_stg2/image_08003.jpg,0.241881,0.037752,0.007778,0.007778,0.393834,0.059190,0.021995,0.237437
5,test_stg2/image_05669.jpg,0.344554,0.074545,0.012542,0.007778,0.262711,0.062882,0.024263,0.210736
6,test_stg2/image_11511.jpg,0.574056,0.030472,0.007778,0.007778,0.014284,0.075149,0.026620,0.270510
7,test_stg2/image_10794.jpg,0.007778,0.007778,0.007778,0.007778,0.124734,0.015764,0.045092,0.807639
8,test_stg2/image_04845.jpg,0.230849,0.007778,0.007778,0.007778,0.429894,0.011407,0.007778,0.320754
9,test_stg2/image_02907.jpg,0.732523,0.012688,0.007778,0.007778,0.238167,0.007778,0.007778,0.010752


In [177]:
results.to_csv(subm_csv, index=False)

In [153]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [154]:
save_array(path+'results/Fisheries-Model-CNN-Results.dat', predictions)